In [ ]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

from qiskit import *
from math import pi
from qiskit.visualization import *
from math import *

# import basic plot tools
from qiskit.visualization import plot_histogram

In [ ]:
# Let's define the oracle
def minttwo(nqubits):
    # nqubits should be 3
    qc = QuantumCircuit(nqubits)
    
    # Apply 1st and 3rd
    qc.h(2)
    qc.mct([0], 2)  # multi-controlled-toffoli
    qc.h(2)
    
    # Apply 1st and 2nd
    qc.h(2)
    qc.mct([1], 2)  # multi-controlled-toffoli
    qc.h(2)
        
    # We will return the oracle as a gate
    U_w = qc.to_gate()
    U_w.name = "U$_w$"
    return U_w

In [ ]:
# Let's define the diffusion operator
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate: equivalent to multi-controlled toffoli sandwished between H gates
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

In [ ]:
# This is the full algorithm
var_qubits = QuantumRegister(3, name='v')
cbits = ClassicalRegister(3, name='cbits')
qc = QuantumCircuit(var_qubits, cbits)

# Initialize qubits in state |s>
qc.h(var_qubits)

# Apply our oracle
qc.append(minttwo(3), [0,1,2])

# Apply our diffuser
qc.append(diffuser(3), [0,1,2])

# Visual Separation
qc.barrier()

# Measure the variable qubits
qc.measure(var_qubits, cbits)

qc.draw(fold=-1)

In [ ]:
# Simulate and plot results
aer_simulator = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_simulator)
qobj = assemble(transpiled_qc)
result = aer_simulator.run(qobj).result()
plot_histogram(result.get_counts())

# Actual quantum computer to get results
#provider = IBMQ.load_account()
#backend = provider.get_backend('ibmq_manila')
#results = execute(qc,backend,shots=1024).result().get_counts()
#plot_histogram(results)

Answers found by Grover's are: 101 and 110 since they have the highest measurement probabilities.